<a href="https://colab.research.google.com/github/ArjunaBazaz/DS2002ChatBot/blob/main/ChatBotHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Step 0: Ensure TensorFlow 2.x is installed
# !pip install tensorflow

# Step 1: Import necessary libraries

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np


In [21]:
questions = [
    'Hello',
    'How are you?',
    'What is your name?',
    'What date was this assignment submitted',
    'What is your favorite color?',
    'What is TensorFlow?',
    'Do you like Python?',
    'What is the capital of the USA?',
    'How many classes do you have on Monday?',
    'How many classes do you have on Friday?',
    'What do you like to do in your free time?',
    'Who wrote Hamlet?',
    'What is quantum computing?',
    'What is the speed of light?',
    'Can you solve math problems?',
    'What is your favorite movie?',
    'Do you know Siri?',
    'Can you play music?',
    'How do I make tea?',
    'What’s the weather like?',
    'Tell me something interesting.',
    'Who created you?',
    'Do you dream of electric sheep?',
    'Can you speak other languages?',
    'What is machine learning?',
    'Who won the World Cup in 2018?',
]

answers = [
    'Hello, I hope you are having a great day',
    'I am just a bot, but I am doing fine.',
    'I am ArjunaBot created by Arjuna Bazaz for his data science class.',
    'This assignment was submitted on 4/6/2024',
    'My favorite color is purple because that is the favorite color of the one who created me',
    'TensorFlow is a machine learning library.',
    'Yes, Python is great!',
    'The Capital of the USA is Washington DC',
    'I have 2 classes on Monday',
    'I have 1 class on Friday',
    'I spend my free time at the gym, playing basketball, or studying',
    'William Shakespeare wrote Hamlet.',
    'Quantum computing is computing using quantum-mechanical phenomena.',
    'The speed of light is approximately 299,792 kilometers per second.',
    'I can help solve simple math problems.',
    'I do not watch movies, but I can discuss them based on my training data.',
    'Yes, Siri is another virtual assistant.',
    'I cannot play music, but I can chat with you.',
    'Boil water, then pour it over your tea. Steep for a few minutes.',
    'I cannot check real-time data, including the weather.',
    'The shortest war in history lasted 38 minutes.',
    'I was created by developers using TensorFlow and Python.',
    'I think about learning data patterns, not sheep.',
    'Yes, but I am primarily programmed to respond in English.',
    'Machine learning is a field of AI focused on teaching machines to learn from data.',
    'France won the FIFA World Cup in 2018.',
]

print(len(questions), len(answers))

26 26


In [22]:
# Example dataset: A small set of question-answer pairs
#questions = ['How are you?', 'What is your name?', 'What is TensorFlow?', 'Do you like Python?']
#answers = ['I am fine.', 'I am a bot.', 'TensorFlow is a machine learning library.', 'Yes, Python is great!']

# Tokenize the sentences (this is a very simplified approach)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)

# Pad the sequences
max_length = max(max(len(seq) for seq in sequences_questions), max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding='post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1


In [23]:
# Define model parameters
embedding_dim = 256
units = 1024

# Define the encoder model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder model
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_10 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_8 (Embedding)     (None, None, 256)            44544     ['input_9[0][0]']             
                                                                                                  
 embedding_9 (Embedding)     (None, None, 256)            44544     ['input_10[0][0]']            
                                                                                            

In [25]:
# Prepare decoder input data that just contains the start token and the rest is all 0
# It's a simplified approach, in practice, you would use teacher forcing or another technique
decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1  # Assuming 1 is the start token

# Train the model
model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=100)


Epoch 1/100
13/13 [==============================] - 0s 17ms/step - loss: 0.9112 - accuracy: 0.6267
Epoch 2/100
13/13 [==============================] - 0s 14ms/step - loss: 1.0499 - accuracy: 0.5792
Epoch 3/100
13/13 [==============================] - 0s 14ms/step - loss: 1.1431 - accuracy: 0.6041
Epoch 4/100
13/13 [==============================] - 0s 13ms/step - loss: 0.8546 - accuracy: 0.6222
Epoch 5/100
13/13 [==============================] - 0s 13ms/step - loss: 1.1449 - accuracy: 0.6244
Epoch 6/100
13/13 [==============================] - 0s 13ms/step - loss: 0.7335 - accuracy: 0.6946
Epoch 7/100
13/13 [==============================] - 0s 13ms/step - loss: 0.7603 - accuracy: 0.6900
Epoch 8/100
13/13 [==============================] - 0s 14ms/step - loss: 0.8526 - accuracy: 0.6878
Epoch 9/100
13/13 [==============================] - 0s 13ms/step - loss: 0.6998 - accuracy: 0.6900
Epoch 10/100
13/13 [==============================] - 0s 13ms/step - loss: 0.9015 - accuracy: 0.6697

In [26]:
# Step 5: Implement the chat function
def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1  # start token
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:  # end token
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

# Step 6: Chat with the bot
input_text = "What is TensorFlow?"
print(f"You: {input_text}")
print(f"Bot: {chat_with_bot(input_text)}")

You: What is TensorFlow?
1/1 [==============================] - 0s 22ms/step
Bot: tensorflow a machine learning library


In [ ]:
# Interactive chat with the bot
print("Start chatting with the bot! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit' or user_input.lower() == 'goodbye' or user_input.lower() == "i'm done":
        break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")


Start chatting with the bot! Type 'quit' to exit.
You: what is your name
1/1 [==============================] - 0s 24ms/step
Bot: 
You: What is Tensorflow?
1/1 [==============================] - 0s 25ms/step
Bot: tensorflow a machine learning library
You: What's the weather like?
1/1 [==============================] - 0s 26ms/step
Bot: 
You: Do you know Siri
1/1 [==============================] - 0s 31ms/step
Bot: yes siri another virtual assistant
